## Testing some python codes

# Load standard behavior and neural data

In [1]:
# cd to neuro-behavior/src directory, and add the figure_tools package folder to path
import os
os.chdir('E:/Projects/neuro-behavior/src/')
import sys
sys.path.append('E:/Projects/')
import figure_tools as ft
from neurobehavior.neuro_behavior_options import neuro_behavior_options
import matplotlib.pyplot as plt


In [2]:
# make any options changes
opts = neuro_behavior_options()
opts['frameSize'] = .05
opts['collectFor'] = 45 * 60

%run -i get_standard_data.py

-1: in_nest_sleeping_or_irrelevant: Valid: 177 (72.5)%

0: investigate_1: Valid: 202 (78.9)%

1: investigate_2: Valid: 449 (48.4)%

2: investigate_3: Valid: 83 (71.6)%

3: rear: Valid: 26 (81.2)%

4: dive_scrunch: Valid: 97 (85.1)%

5: paw_groom: Valid: 66 (66.0)%

6: face_groom_1: Valid: 127 (60.5)%

7: face_groom_2: Valid: 74 (54.8)%

8: head_groom: Valid: 91 (65.0)%

9: contra_body_groom: Valid: 47 (65.3)%

10: ipsi_body groom: Valid: 67 (59.8)%

11: contra_itch: Valid: 113 (66.1)%

12: ipsi_itch_1: Valid: 85 (63.0)%

13: contra_orient: Valid: 248 (71.1)%

14: ipsi_orient: Valid: 332 (67.5)%

15: locomotion: Valid: 500 (49.9)%


Keeping 225 of 499 neurons
20 M23
49 M56
96 DS
60 VS


E:\Projects\neuro-behavior\src\get_standard_data.py:127: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  dataBhv.StartFrame = 1 + np.floor(dataBhv.StartTime / opts['frameSize'])
E:\Projects\neuro-behavior\src\get_standard_data.py:128: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  dataBhv.DurFrame = np.floor(dataBhv.Dur / opts['frameSize'])


# Pacmap
[github link](https://github.com/YingfanWang/PaCMAP)

In [4]:
idInd = idDS

In [4]:
import pacmap

X = dataMat[:, idDS]
# X = X.reshape(X.shape[0], -1)
y = bhvIDMat

# initializing the pacmap instance
# Setting n_neighbors to "None" leads to a default choice shown below in "parameter" section
embedding = pacmap.PaCMAP(n_components=2, n_neighbors=None, MN_ratio=0.5, FP_ratio=2.0) 

# fit the data (The index of transformed data corresponds to the index of the original data)
X_transformed = embedding.fit_transform(X, init="pca")

### Visualize Pacmap

In [18]:
%matplotlib qt5
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

bhvColors = ft.colors_for_behaviors(behaviors)
# Prepare colors: Map each label to its RGB color
bhvIDMatColors = bhvIDMat + 1
colors = np.array([bhvColors[label] for label in bhvIDMatColors])  # Adjust indexing if necessary

# The third dimension (Z) is a sequence from 1 to dataMat.shape[0]
z = np.arange(1, dataMat.shape[0] + 1)

# visualize the embedding
fig = plt.figure(figsize=(18, 18))
ax = fig.add_subplot(111, projection='3d')

ax.scatter(X_transformed[:, 0], X_transformed[:, 1], z, color=colors, s=10, 
           marker='o', linewidths=1.5, facecolors='none')
ax.set_title('PACMAP projection of the dataMat dataset', fontsize=24)
ax.set_xlabel('Component 1')
ax.set_ylabel('Component 2')
plt.tight_layout()

plt.show()


# UMAP

In [6]:
import umap

# Initialize UMAP. You can adjust the parameters as needed.
reducer = umap.UMAP(n_neighbors=15, n_components=2, metric='euclidean')

# Fit the model to your data
embedding = reducer.fit_transform(dataMat[:, idInd])

NameError: name 'idInd' is not defined

### Visualize UMAP

In [ ]:
import matplotlib.pyplot as plt

bhvColors = ft.colors_for_behaviors(behaviors)
# Prepare colors: Map each label to its RGB color
bhvIDMatColors = bhvIDMat + 1
colors = np.array([bhvColors[label] for label in bhvIDMatColors])  # Adjust indexing if necessary

# The third dimension (Z) is a sequence from 1 to dataMat.shape[0]
z = np.arange(1, dataMat.shape[0] + 1)

# Plotting the result
fig = plt.figure(figsize=(18, 18))
ax = fig.add_subplot(111, projection='3d')

ax.scatter(embedding[:, 0], embedding[:, 1], z, color=colors, s=10, 
           marker='o', linewidths=1.5, facecolors='none')
ax.set_title('UMAP projection of the dataMat dataset', fontsize=24)
ax.set_xlabel('Component 1')
ax.set_ylabel('Component 2')

plt.tight_layout()
plt.show()


Get unique sequences


In [5]:
from neurobehavior.find_unique_sequences import find_unique_sequences

nSeq = 3
validInd = [0, 1, 0]
uniqueSequences, sequenceIndices = find_unique_sequences(dataBhv, nSeq, validInd)
print("Unique Sequences:", uniqueSequences)
# print("Sequence Indices:", sequenceIndices)
print("Number of sequences:", [len(sublist) for sublist in sequenceIndices])

Unique Sequences: [[15, 1, 15], [1, 15, 1], [14, 15, 1], [1, 14, 15], [15, 14, 15], [13, 15, 1], [14, 15, 14], [1, 15, 14], [15, 1, 13], [1, 13, 15], [15, 1, 14], [1, 0, 1], [1, 13, 1], [13, -1, 15], [15, 13, 15], [0, 14, 15], [15, 13, 1], [1, 15, 13], [15, 14, 1], [14, 1, 15], [1, -1, 1], [7, 8, 7], [13, 15, 13], [13, 1, 15], [1, 14, 1], [-1, 15, 1], [15, 0, 14], [1, 15, 0], [15, 0, 1], [14, 0, 14], [14, 2, 14], [-1, 1, 15], [11, 9, 8], [11, 9, 11], [15, 0, 13], [15, 13, -1], [7, 6, 7], [1, 13, -1], [1, 14, 0], [0, 14, 0], [6, 5, 6], [12, 15, 12], [14, 2, 1], [0, 1, 0], [0, 14, 1], [14, 15, 0], [1, 14, 2], [14, 2, 4], [12, 15, 14], [10, 12, 10], [15, 1, -1], [-1, 15, 13], [12, 15, 1], [15, 1, 0], [11, 15, 13], [10, -1, 10], [11, 13, 15], [15, -1, 15], [4, 13, 15], [14, 5, 14], [14, 1, 14], [0, 13, 1], [2, 14, 15], [6, -1, 6], [1, 15, -1], [0, 1, 13], [13, 1, 14], [13, 1, -1], [1, -1, 15], [9, 11, 9], [0, 13, 15], [6, 0, 6], [8, 11, 9], [-1, 10, 12], [0, 1, 15], [15, 0, 15], [14, 0, 1]

Get only the triplets with a particular behavior in the middle

In [7]:
middleBhv = 6

# Find indices of sequences in uniqueSequences where the second element equals middleBhv
middleIdx = [i for i, seq in enumerate(uniqueSequences) if seq[1] == middleBhv]

# Retrieve the sequences corresponding to middleIdx
selectedSequences = [uniqueSequences[i] for i in middleIdx]

# Calculate and display the lengths of the sequence indices corresponding to middleIdx
lengths = [len(sequenceIndices[i]) for i in middleIdx]

# Display the selected sequences and lengths
print(selectedSequences)
print(lengths)


[[7, 6, 7], [5, 6, 5], [8, 6, 8], [4, 6, 10], [15, 6, 14], [4, 6, 0], [15, 6, 0], [14, 6, 14], [14, 6, 0], [14, 6, 8], [4, 6, 5], [0, 6, 0], [1, 6, 8], [14, 6, 12], [15, 6, 8], [15, 6, 15], [13, 6, 13], [13, 6, 7], [0, 6, 5], [1, 6, 7], [1, 6, 1], [7, 6, 1], [1, 6, 14], [0, 6, 7], [4, 6, 14], [15, 6, 13], [2, 6, 4], [0, 6, 14], [0, 6, 4], [15, 6, 12], [13, 6, 5], [14, 6, 15], [11, 6, 13], [10, 6, -1], [13, 6, 14], [5, 6, 4], [-1, 6, -1], [10, 6, 11], [1, 6, 5], [-1, 6, 7], [7, 6, 15], [13, 6, 8], [14, 6, 13], [11, 6, 1], [8, 6, 0], [11, 6, -1], [4, 6, 12], [14, 6, 4], [15, 6, 10], [-1, 6, 13], [8, 6, 7], [13, 6, -1], [14, 6, 7], [14, 6, 5], [12, 6, 0], [-1, 6, 5], [5, 6, 0], [5, 6, 13], [8, 6, -1], [0, 6, 2], [5, 6, 15], [15, 6, 7], [15, 6, -1], [2, 6, 2], [10, 6, 14], [14, 6, 10], [14, 6, 1]]
[11, 6, 6, 5, 4, 4, 4, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

Project the sequence data into the UMAP space

In [ ]:
# Project new data
new_embedding = reducer.transform(new_data)